In [1]:
from CDM_DataSetMaker_ops import get_conn
from RETAIN_Dataset import convert_transaction_to_seq

conn = get_conn('./DB_connection.txt')
#T_seq = convert_transaction_to_seq(conn, 'condition_concept_id', 'HF_T_dx.pkl')
#C_seq = convert_transaction_to_seq(conn, 'condition_concept_id', 'HF_C_dx.pkl')

SERVER_IP: "128.1.99.53:1433"
USER: "yrlee"
PASSWD: "qwer1234!@"
DB: "LJ_HF_replica_study"
Done 13.871742010116577
Dumping at.. ./HF_C_dx.pkl


In [1]:
from CDM_DataSetMaker_ops import loadingFiles

T_seq = loadingFiles('./', 'HF_T_dx.pkl')
C_seq = loadingFiles('./', 'HF_C_dx.pkl')

len(T_seq), len(C_seq)

Loading at.. ./HF_T_dx.pkl
Loading at.. ./HF_C_dx.pkl


(22548, 85011)

In [2]:
from RETAIN_Dataset_Final import truncated_data, code_to_id

HF_T_cut = truncated_data(T_seq, cut_num=2)
HF_C_cut = truncated_data(C_seq, cut_num=2)
HF_code_dict, max_visit_size = code_to_id(HF_T_cut+HF_C_cut)

code_size:  5088


In [3]:
from RETAIN_Dataset_Final import RETAIN_datasets

datasets = RETAIN_datasets(HF_T_cut, HF_C_cut, HF_code_dict, max_visit_size)

In [4]:
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
batch_size = 10
time_size = max_visit_size
code_size = len(HF_code_dict)
label_size = 2
lr_init = 0.0001
decay_step = 2000
decay_rate = 0.9
training_step = 5000
printby = 100

hidden_size_alpha = 128
hidden_size_beta = 128

In [6]:
import tensorflow as tf

def sequence_masking(data, visit_times):
    masking = tf.tile(tf.reshape(tf.sequence_mask(visit_times, data.shape[1]), shape=[-1,data.shape[1],1]), [1,1,data.shape[2]])
    return tf.where(masking, data, tf.zeros_like(data))

In [7]:
inputs = tf.placeholder(tf.float32, shape=[None, time_size, code_size])
labels = tf.placeholder(tf.float32, shape=[None, time_size, label_size])
visit_times = tf.placeholder(tf.float32, shape=[None])
global_step = tf.Variable(0, trainable=False, dtype=tf.int32)
lr = tf.train.exponential_decay(lr_init, global_step, decay_step, decay_rate, staircase=True)

##embedding

##alpha
W_alpha = tf.Variable(tf.random_normal(shape=[hidden_size_alpha, 1], name='W_alpha'))
b_alpha = tf.Variable(tf.random_normal(shape=[], name='b_alpha'))
with tf.variable_scope('alpha'):
    cells_alpha = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(hidden_size_alpha)])
    outputs_alpha, states_alpha = tf.nn.dynamic_rnn(cells_alpha, inputs, visit_times, dtype=tf.float32)
    reshaped_outputs_alpha = tf.reshape(outputs_alpha, shape=[-1, hidden_size_alpha])
    matmuled_alpha = tf.matmul(reshaped_outputs_alpha, W_alpha) + tf.expand_dims(b_alpha, 0)
    reshaped_matmuled_alpha = tf.reshape(matmuled_alpha, shape=[-1, time_size, 1])
    logits_alpha = tf.nn.softmax(reshaped_matmuled_alpha, 1)
    
##Beta
W_beta = tf.Variable(tf.random_normal(shape=[hidden_size_beta, code_size]))
b_beta = tf.Variable(tf.random_normal(shape=[code_size]))
with tf.variable_scope('beta'):
    cells_beta = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(hidden_size_beta)])
    outputs_beta, states_beta = tf.nn.dynamic_rnn(cells_beta, inputs, visit_times, dtype=tf.float32)
    reshaped_outputs_beta = tf.reshape(outputs_beta, shape=[-1, hidden_size_beta])
    matmuled_ouputs_beta = tf.matmul(reshaped_outputs_beta, W_beta) + tf.expand_dims(b_beta, 0)
    reshaped_matmuled_beta = tf.reshape(matmuled_ouputs_beta, shape=[-1, time_size, code_size])
    logits_beta = tf.nn.tanh(reshaped_matmuled_beta)
    
##unifiy
W_s = tf.Variable(tf.random_normal(shape=[code_size, label_size]))
b_s = tf.Variable(tf.random_normal(shape=[label_size]))
unified_context = logits_alpha*logits_beta*inputs
#context_vec = []
#for b in range(batch_size):
#    pid_seq = unified_context[b]
#    for t in range(time_size):
#        context_vec.append(tf.reduce_sum(pid_seq[:t+1, :], axis=0))
reshaped_context_vec = tf.reshape(unified_context, shape=[-1, code_size])
mlp_context_vec = tf.matmul(reshaped_context_vec, W_s) + b_s
logits = tf.nn.softmax(tf.reshape(mlp_context_vec, shape=[-1, time_size, label_size]))
masked_logits = sequence_masking(logits, visit_times)
masked_labels = sequence_masking(labels, visit_times)

##Loss function
loss_per_times = tf.reduce_sum(masked_labels*tf.log(masked_logits+1e-10)+(1-masked_labels)*tf.log(1-masked_logits+1e-10), axis=-1)
loss_per_pid = tf.reduce_sum(loss_per_times, axis=-1)*(1/visit_times)
last_loss = -tf.reduce_sum(loss_per_pid)*(1/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(last_loss, global_step=global_step)

##accuracy
#argmax_logits = tf.argmax(tf.reshape(masked_logits, shape=[-1,label_size]), axis=1)
#argmax_labels = tf.argmax(tf.reshape(masked_labels, shape=[-1,label_size]), axis=1)
#compared = tf.cast(tf.equal(argmax_logits, argmax_labels), tf.float32)
#accuracy = tf.reduce_mean(compared, axis=0)

In [8]:
from tqdm import trange
with tf.Session() as sess:
    tf.train.start_queue_runners(sess=sess)
    tf.global_variables_initializer().run(session=sess)

    for step in trange(training_step):
        batch_train_inputs, batch_train_labels, batch_train_visit_times = datasets.train.next_batch(batch_size)
        batch_val_inputs, batch_val_labels, batch_val_visit_times = datasets.validation.next_batch(batch_size)
        batch_te_inputs, batch_te_labels, batch_te_visit_times = datasets.test.next_batch(batch_size)
        train_feed_dict = {inputs: batch_train_inputs, 
                           labels: batch_train_labels, 
                           visit_times: batch_train_visit_times}
        validation_feed_dict = {inputs: batch_val_inputs, 
                                labels: batch_val_labels, 
                                visit_times: batch_val_visit_times}
        test_feed_dict = {inputs: batch_te_inputs, 
                          labels: batch_te_labels, 
                          visit_times: batch_te_visit_times}
        _, g_step, train_loss, train_lr_ = sess.run([optimizer, global_step, last_loss, lr], feed_dict=train_feed_dict)
        val_loss, val_lr_ = sess.run([last_loss, lr], feed_dict=validation_feed_dict)
        
        if step % printby == 0:
            print('step: {} \tg_step: {} \tloss: {:.8f}||{:.8f} \tlr: {:.8f}||{:.8f}'.format(step, g_step, train_loss, val_loss, train_lr_, val_lr_))
        
        if step == training_step-1:
            te_loss = sess.run(last_loss, feed_dict=test_feed_dict)
            print('All Done! test_loss is {}'.format(te_loss))

  0%|          | 1/5000 [00:05<7:06:45,  5.12s/it]

step: 0 	g_step: 1 	loss: 1.43568647||1.44257545 	lr: 0.00010000||0.00010000


  2%|▏         | 101/5000 [06:25<5:04:26,  3.73s/it]

step: 100 	g_step: 101 	loss: 1.42380190||1.43273389 	lr: 0.00010000||0.00010000


  4%|▍         | 201/5000 [11:54<4:25:27,  3.32s/it]

...

step: 4800 	g_step: 4801 	loss: 1.30168664||1.32947624 	lr: 0.00008100||0.00008100


 98%|█████████▊| 4901/5000 [2:58:15<01:18,  1.26it/s]

step: 4900 	g_step: 4901 	loss: 1.46170807||1.28822505 	lr: 0.00008100||0.00008100


100%|██████████| 5000/5000 [2:59:33<00:00,  1.18it/s]

All Done! test_loss is 1.2397422790527344
